In [ ]:
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from rasterio.enums import Resampling
from PIL import ImageColor
from skimage.exposure import rescale_intensity
import rasterio as rio
import os
import pandas as pd
from PIL import ImageColor
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Conv2D, Conv2DTranspose, Dropout, MaxPooling2D, Input, concatenate
from keras.models import Model
from tensorflow.keras.metrics import MeanIoU, CategoricalAccuracy, Precision, Recall, AUC
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report 
from skimage.exposure import rescale_intensity
from rasterio.enums import Resampling
import rasterio.warp as warp

In [ ]:
#Localización del data: JSON de clases con paleta de colores, imagen original y su máscara.
colores_dir = r'C:\Users\Usuari\Documents\TFM_Codigos\MODELO\data/7Labels.json'
colores = json.load(open(colores_dir))
colores_df = pd.DataFrame(colores)
colores_df["values_normalize"] = colores_df.index
colores_df["palette"] = "#" + colores_df["palette"]
values = colores_df["values"].to_list()
values_norm = colores_df["values_normalize"].to_list()
palette = colores_df["palette"].to_list()
labels = colores_df["label"].to_list()
dict_values = {}
dict_label = {}
dict_palette = {}
dict_palette_hex = {}
for x in range(0, len(values)):
    dict_values[values[x]] = values_norm[x]
    dict_label[values_norm[x]] = labels[x]
    dict_palette[values_norm[x]] = ImageColor.getrgb(palette[x])
    dict_palette_hex[values_norm[x]] = palette[x]
cmap = ListedColormap(palette)
patches = [
    mpatches.Patch(color=palette[i], label=labels[i]) for i in range(len(values))
]
legend = {
    "handles": patches,
    "bbox_to_anchor": (1.05, 1),
    "loc": 2,
    "borderaxespad": 0.0,
}
colores_df

In [ ]:
import numpy as np
import os
import re

def load_and_combine_datasets(folder_path):
    files = os.listdir(folder_path)
    image_train_pattern = re.compile(r".npy") 
    image_test_pattern = re.compile(r".npy")
    mascara_train_pattern = re.compile(r".npy")
    mascara_test_pattern = re.compile(r".npy")
    
    #Almacenar los arrays cargados
    image_train_arrays = []
    image_test_arrays = []
    mascara_train_arrays = []
    mascara_test_arrays = []
    
    for file in files:
        if image_train_pattern.match(file):
            image_train_arrays.append(np.load(os.path.join(folder_path, file)))
        elif image_test_pattern.match(file):
            image_test_arrays.append(np.load(os.path.join(folder_path, file)))
        elif mascara_train_pattern.match(file):
            mascara_train_arrays.append(np.load(os.path.join(folder_path, file)))
        elif mascara_test_pattern.match(file):
            mascara_test_arrays.append(np.load(os.path.join(folder_path, file)))
    combined_images_train = np.concatenate(image_train_arrays, axis=0)
    combined_images_test = np.concatenate(image_test_arrays, axis=0)
    combined_mascara_train = np.concatenate(mascara_train_arrays, axis=0)
    combined_mascara_test = np.concatenate(mascara_test_arrays, axis=0)
    
    return combined_images_train, combined_images_test, combined_mascara_train, combined_mascara_test

folder_path = r"C:\Users\Usuari\Documents\TFM_Codigos\DATASET\Data-arrays"
images_train, images_test, mascara_train, mascara_test = load_and_combine_datasets(folder_path)

In [ ]:
mascara_train_category = to_categorical(mascara_train)
mascara_test_category = to_categorical(mascara_test)

In [ ]:
input_shape = images_train.shape
input_shape = (input_shape[1], input_shape[2], input_shape[3])
num_classes = 7
neuron = 64
kernel = 3
kernel_t = 2
dropout = 0.2
strides = 2
pool = 2
padding = 'same'
input_layer = Input(input_shape)
def conv2_block(input, neuron, last=False):
	conv1 = Conv2D(neuron, kernel, activation='relu', padding=padding)(input)
	conv2 = Conv2D(neuron, kernel, activation='relu', padding=padding)(conv1)
	return conv2
def encode(input, neuron):
	conv1 = conv2_block(input, neuron)
	mp = MaxPooling2D(2)(conv1)
	dp = Dropout(dropout)(mp)
	return conv1, dp
def decode(input, conv, neuron):
	conv_t = Conv2DTranspose(neuron, kernel_t, strides, activation='relu', padding=padding)(input)
	concat = concatenate([conv_t, conv])
	conv2 = conv2_block(concat, neuron)
	dp = Dropout(dropout)(conv2)
	return dp
conv1, mp1 = encode(input_layer, neuron * 1)
conv2, mp2 = encode(mp1, neuron * 2)
conv3, mp3 = encode(mp2, neuron * 4)
conv4, mp4 = encode(mp3, neuron * 8)
transition = conv2_block(mp4, neuron * 16)
uncov1 = decode(transition, conv4, neuron * 8)
uncov2 = decode(uncov1, conv3, neuron * 4)
uncov3 = decode(uncov2, conv2, neuron * 2)
uncov4 = decode(uncov3, conv1, neuron * 1)
output = Conv2D(mascara_train_category.shape[3], 1, padding=padding, activation='softmax')(uncov4)

model = Model(input_layer, output)
model.summary()
mascaras_train_category = 1

In [ ]:
model.compile(
	optimizer=keras.optimizers.Adam(0.001),  
	loss=keras.losses.CategoricalCrossentropy(),
	metrics=[keras.metrics.CategoricalAccuracy(), keras.metrics.MeanIoU(num_classes=num_classes), Recall(), Precision()	
])
callbacks = [
	EarlyStopping(patience=5, monitor='categorical_accuracy', restore_best_weights=True)
]
result = model.fit(
	x=images_train,
	y=mascaras_train_category,
	epochs= 35,
	batch_size= 8, #16
	shuffle=True,
	validation_split=0.2,
	callbacks=callbacks,
    verbose=1,
)

Resultados

In [ ]:
history = pd.DataFrame(result.history)
print(history)
plt.figure(figsize = (10, 8))
plt.plot(range(len(history['categorical_accuracy'].values.tolist())), history['categorical_accuracy'].values.tolist(), label = 'Train_Accuracy')
plt.plot(range(len(history['loss'].values.tolist())), history['loss'].values.tolist(), label = 'Train_Loss')
plt.plot(range(len(history['val_categorical_accuracy'].values.tolist())), history['val_categorical_accuracy'].values.tolist(), label = 'Test_Accuracy')
plt.plot(range(len(history['val_loss'].values.tolist())), history['val_loss'].values.tolist(), label = 'Test_Loss')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
# Predict test data
prediction = np.argmax(model.predict(images_test), 3).flatten()
label = mascara_test.flatten()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = model.predict(images_test)  
y_pred = np.argmax(y_pred, axis=-1)  
y_true = np.argmax(mascara_test_category, axis=-1)  
y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()
# Calcular la matriz de confusión
cm = confusion_matrix(y_true_flat, y_pred_flat, labels=np.arange(num_classes))
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Verdadero')
plt.show()

In [ ]:
# Normalizar la matriz de confusión (entre 0 y 1)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=labels, yticklabels=labels, cbar_kws={'label': 'Frecuencia Relativa'})
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
cm = confusion_matrix(label, prediction, normalize='true')
fig, ax = plt.subplots(figsize=(15, 10))
cm = ConfusionMatrixDisplay(cm)
cm.plot(ax = ax)
print(classification_report(label, prediction))

In [ ]:
image_non_augment_size = int(len(images_test) / 8)

plt.figure(figsize=(8, image_non_augment_size * 2))
for x in range(0, image_non_augment_size - 1):
	index = x * 8

	image = images_test[index:(index + 1)]
	pred = model.predict(image)
	pred = np.argmax(pred, 3)[0]

	plt.subplot(image_non_augment_size, 3, x * 3 + 1)
	plt.imshow(image[0])

	plt.subplot(image_non_augment_size, 3, x * 3 + 2)
	plt.imshow(mascara_test[index], cmap=cmap, interpolation='nearest', vmin=0, vmax=6)

	plt.subplot(image_non_augment_size, 3, x * 3 + 3)
	plt.imshow(pred, cmap=cmap, interpolation='nearest', vmin=0, vmax=6)

In [ ]:
model.save('UNET40_7L_Javi_v03042025.keras')